In [1]:
%load_ext cython
import matplotlib.pyplot as plt
import meshpy.tet
import numpy as np
#%load_ext snakeviz

In [2]:
%%cython -a
import numpy as np
def build_aabb(int nx, int ny, int nz, long [:,:] tetra, double [:,:] nodes, long [:,:] aabb, double [:] step):
    cdef int i, ntetra, j, ix, iy, iz
    cdef double [:,:] bb = np.zeros((2,3))
#     cdef double [:] maxs = np.zeros(3)
    cdef double [:,:] points = np.zeros((4,3))
    ntetra = tetra.shape[0]
    # create a mask for extracting corners from 
    cdef long [:,:] corners = np.array([[0, 1, 0, 0, 1, 0, 1, 1],
                                        [0, 0, 1, 0, 0, 1, 1, 1],
                                        [0, 0, 0, 1, 1, 1, 0, 1]]).astype(int)
    
    cdef long [:] tetra_count= np.zeros(nx*ny*nz).astype(int)
    for i in range(ntetra):
        # copy points into tmp array
        for k in range(4):
            for l in range(3):
                points[k,l] = nodes[tetra[i,k],l]
        for l in range(3):
            bb[0,l] = np.min(points[:,l])#,axis=0)
            bb[1,l] = np.max(points[:,l])#,axis=0)
#             print(bb[0,l],bb[1,l])
        # check which corners of bounding box are inside a cell 
        for j in range(8):
            ix = int(bb[corners[0,j],0] // step[0]) - 1
            iy = int(bb[corners[1,j],1] // step[1]) - 1
            iz = int(bb[corners[2,j],2] // step[2]) - 1

            aabb[ix+nx*iy+nx*ny*iz,tetra_count[ix+nx*iy+nx*ny*iz]] = i
            tetra_count[ix+nx*iy+nx*ny*iz]+=1


        
    return

In [3]:
%%cython -a
cdef stp(double [:] v1, double [:] v2, double [:] v3):
    cdef double [3] c

    s = 0.
    c[0] = v1[1]*v2[2]-v1[2]*v2[1]
    c[1] = v1[2]*v2[0]-v1[0]*v2[2]
    c[2] = v1[0]*v2[1]-v1[1]*v2[0]
    s = c[0]*v3[0] + c[1]*v3[1] + c[2]*v3[2]
    return s
import numpy as np
def find_tetras(double [:,:] nodes, long [:,:] tetras, long [:,:] aabbindx, double [:,:] pos, 
               double [:,:] bcs, long [:] tetra_index):
    cdef int npts = pos.shape[0]
    cdef int ntetra 
    cdef int i,j,k, l
    cdef double [3] vap, vbp, vab, vac, vad, vbc, vbd
    cdef double va, vb, vc, vd, v
    cdef double [4] c
    for i in range(npts):
        ntetra = len(aabbindx[i,:])
        for j in range(ntetra):
            if aabbindx[i,j] < 0:
                continue
            for k in range(3):
                vap[k] = pos[i,k] - nodes[tetras[aabbindx[i,j],0],k]
                vbp[k] = pos[i,k] - nodes[tetras[aabbindx[i,j],1],k]
                
                vab[k] = nodes[tetras[aabbindx[i,j],1],k] - nodes[tetras[aabbindx[i,j],0],k]
                vac[k] = nodes[tetras[aabbindx[i,j],2],k] - nodes[tetras[aabbindx[i,j],0],k]
                vad[k] = nodes[tetras[aabbindx[i,j],3],k] - nodes[tetras[aabbindx[i,j],0],k]
                vbc[k] = nodes[tetras[aabbindx[i,j],2],k] - nodes[tetras[aabbindx[i,j],1],k]
                vbd[k] = nodes[tetras[aabbindx[i,j],3],k] - nodes[tetras[aabbindx[i,j],1],k]
                
            va = 0
            vb = 0
            vc = 0
            vd = 0
            v = 0
            # calculate scalar triple product as each bc coordinate
            # if any coordinte is 0 then this tetra is not the right one so keep looking
            v = stp(vac,vad,vab)
            v /= 6.
            if v <= 0:
                continue
            va = stp(vbd,vbc,vbp)
            va /= 6.
            va /= v
            if va < 0:
                continue
            vb = stp(vac,vad,vap)
            vb /= 6.
            vb /= v
            if vb < 0:
                continue
            vc = stp(vad,vab,vap)
            vc /= 6.
            vc /= v
            if vc < 0:
                continue
            vd = stp(vab,vac,vap)
            vd /= 6.
            vd /= v
            if vd < 0:
                continue

            # all three coordinates should be > 0
            tetra_index[i] = aabbindx[i,j]
            bcs[i,0] = va
            bcs[i,1] = vb
            bcs[i,2] = vc
            bcs[i,3] = vd

                
#                 vab = points[:, :, 1, :] - points[:, :, 0, :]
#                 vac = points[:, :, 2, :] - points[:, :, 0, :]
#                 vad = points[:, :, 3, :] - points[:, :, 0, :]
#                 vbc = points[:, :, 2, :] - points[:, :, 1, :]
#                 vbd = points[:, :, 3, :] - points[:, :, 1, :]
#             vap = pos[i,:] - 
#         vap = pos[i,:] - nodes[] 
#         vap = pos[:,None,:] - points[:, :, 0, :]
#         vbp = pos[:,None,:] - points[:, :, 1, :]
#         # vcp = p - points[:, 2, :]
#         # vdp = p - points[:, 3, :]
#         vab = points[:, :, 1, :] - points[:, :, 0, :]
#         vac = points[:, :, 2, :] - points[:, :, 0, :]
#         vad = points[:, :, 3, :] - points[:, :, 0, :]
#         vbc = points[:, :, 2, :] - points[:, :, 1, :]
#         vbd = points[:, :, 3, :] - points[:, :, 1, :]
        
#         va = np.einsum('ikj, iklj->ikl', vbp, np.cross(vbd, vbc, axisa=2, axisb=2)) / 6.
#         vb = np.einsum('ikj, iklj->ikl', vap, np.cross(vac, vad, axisa=2, axisb=2)) / 6.
#         vc = np.einsum('ikj, iklj->ikl', vap, np.cross(vad, vab, axisa=2, axisb=2)) / 6.
#         vd = np.einsum('ikj, iklj->ikl', vap, np.cross(vab, vac, axisa=2, axisb=2)) / 6.
#         v = np.einsum('ikj, iklj->ikl', vab, np.cross(vac, vad, axisa=2, axisb=2)) / 6.
#         c = np.zeros((va.shape[0],va.shape[1],4))

In [4]:
class TetMesh2:
    def __init__(self,origin,maximum,nodes,tetra,neighbours,nx=2,ny=2,nz=2):
        self.origin = np.array(origin)
        self.maximum = np.array(maximum)
        self.nodes = nodes
        self.tetra = tetra
        self.ntetra = tetra.shape[0]

        self.neighbours = neighbours
        self.step = (self.maximum-self.origin)/(np.array([nx,ny,nz]))
        print(self.step)
        self.aabb = np.zeros((nx*ny*nz,self.ntetra)).astype(int)

        self.nx = nx 
        self.nz = nz 
        self.ny = ny 
        
#         nx -= 1
#         ny -=1 
#         nz-=1
        self.aabb[:] = -1

        self._build_aabb()
        self.corners = np.array([[0, 1, 0, 0, 1, 0, 1, 1],
                                      [0, 0, 1, 0, 0, 1, 1, 1],
                                      [0, 0, 0, 1, 1, 1, 0, 1]]).astype(int)
    def _build_aabb(self):

#         print(self.aabb.shape)
#         tetra = self.tetra
#         nodes = self.nodes
#         ntetra = tetra.shape[0]
#         bb = np.zeros((2,3))
#         step = self.step
#         aabb = self.aabb
#         nx = self.nx
#         ny = self.ny
#         nz = self.nz
#         points = np.zeros((4,3))
#         # create a mask for extracting corners from 
#         corners = np.array([[0, 1, 0, 0, 1, 0, 1, 1],
#                                           [0, 0, 1, 0, 0, 1, 1, 1],
#                                           [0, 0, 0, 1, 1, 1, 0, 1]]).astype(int)
#         for i in range(ntetra):
#             # copy points into tmp array
#             for k in range(4):
#                 for l in range(3):
#                     points[k,l] = nodes[tetra[i,k],l]
#             for l in range(3):
#                 bb[0,l] = np.min(points[:,l])#,axis=0)
#                 bb[1,l] = np.max(points[:,l])#,axis=0)

#             for j in range(8):
#                 ix = int(bb[corners[0,j],0] // step[0])
#                 iy = int(bb[corners[1,j],1] // step[1])
#                 iz = int(bb[corners[2,j],2] // step[2])
#                 aabb[ix+nx*iy+nx*ny*iz,i] = i

#         return
        print(self.aabb.shape)
        build_aabb(self.nx,self.ny,self.nz,self.tetra,self.nodes,self.aabb,self.step)
        bb = np.zeros((3,2))
        idx = np.arange(0,self.aabb.shape[0])
        xi,yi,zi = self.global_aabb_index_to_local(idx[np.all(self.aabb == -1,axis=1)])
        bb = np.zeros((3,2))
        for i in range(xi.shape[0]):
            bb[0,0] = xi[i]*self.step[0]
            bb[0,1] = (xi[i]+1)*self.step[0]

            bb[1,0] = yi[i]*self.step[1]
            bb[1,1] = (yi[i]+1)*self.step[1]

            bb[2,0] = zi[i]*self.step[2]
            bb[2,1] = (zi[i]+1)*self.step[2]

            corners = np.array([[0, 1, 0, 0, 1, 0, 1, 1],
                                                [0, 0, 1, 0, 0, 1, 1, 1],
                                                [0, 0, 0, 1, 1, 1, 0, 1]]).astype(int)
            corner_loc = np.zeros((8,3))
            corner_loc[:,0] = bb[0,corners[0,:]]#bb[corners]
            corner_loc[:,1] = bb[1,corners[1,:]]
            corner_loc[:,2] = bb[2,corners[2,:]]
            for j, t in enumerate(self.tetra):
                points = self.nodes[t]
                vap = corner_loc[:,None,:] - points[None,  0, :]
                vbp = corner_loc[:,None,:] - points[None, 1, :]
        #         # vcp = p - points[:, 2, :]
        #         # vdp = p - points[:, 3, :]
                vab = points[None, 1, :] - points[None, 0, :]
                vac = points[None, 2, :] - points[None, 0, :]
                vad = points[None, 3, :] - points[None, 0, :]
                vbc = points[None, 2, :] - points[None, 1, :]
                vbd = points[None, 3, :] - points[None, 1, :]

                va = np.sum(vbp * np.cross(vbd, vbc, axisa=1, axisb=1)) / 6.
                vb = np.sum(vap * np.cross(vac, vad, axisa=1, axisb=1)) / 6.
                vc = np.sum(vap * np.cross(vad, vab, axisa=1, axisb=1)) / 6.
                vd = np.sum(vap * np.cross(vab, vac, axisa=1, axisb=1)) / 6.
                v = np.sum(vab * np.cross(vac, vad, axisa=1, axisb=1)) / 6.

                c = np.zeros((4,8))
                c[0,:] = va / v
                c[1,:] = vb / v
                c[2,:] = vc / v
                c[3,:] = vd / v
#                 print(np.all(c>0,axis=0))
                if np.any(np.all(c>0,axis=0)):
#                     print('All points inside a tetra')
#                     print(self.aabb[xi[i]+yi[i]*self.nx+zi[i]*self.nx*self.ny,0])
                    self.aabb[xi[i]+yi[i]*self.nx+zi[i]*self.nx*self.ny,0] = j
#                     print(self.aabb[xi[i]+yi[i]*self.nx+zi[i]*self.nx*self.ny,0])
                    break
#                 if j == self.
        #         print(mesh2.calc_bary(corner_loc,t))

        #         points = mesh.nodes[t]
        #         print(points)
                if j == len(self.tetra)-1:
                    print(corner_loc)
                    print('Couldnt find any tetra')
    
    
        # now sort
    def get_tetra2(self,pos):
        pos = np.array(pos)
        tetras = np.zeros(pos.shape[0]).astype(int)
        bcs = np.zeros((pos.shape[0],4))
        for i in range(pos.shape[0]):
            aabbidx = self.pos_to_aabb_index(pos)
            cells = self.aabb[aabbidx[i],:]
            cells=cells[cells>=0]
            npts = len(cells)
            points = self.nodes[self.tetra[cells]]
            vap = pos[i,:] - points[:, 0, :]
            vbp = pos[i,:] - points[:, 1, :]
            # vcp = p - points[:, 2, :]
            # vdp = p - points[:, 3, :]
            vab = points[:, 1, :] - points[:, 0, :]
            vac = points[:, 2, :] - points[:, 0, :]
            vad = points[:, 3, :] - points[:, 0, :]
            vbc = points[:, 2, :] - points[:, 1, :]
            vbd = points[:, 3, :] - points[:, 1, :]

            va = np.sum(vbp * np.cross(vbd, vbc, axisa=1, axisb=1), axis=1) / 6.
            vb = np.sum(vap * np.cross(vac, vad, axisa=1, axisb=1), axis=1) / 6.
            vc = np.sum(vap * np.cross(vad, vab, axisa=1, axisb=1), axis=1) / 6.
            vd = np.sum(vap * np.cross(vab, vac, axisa=1, axisb=1), axis=1) / 6.
            v = np.sum(vab * np.cross(vac, vad, axisa=1, axisb=1), axis=1) / 6.
            
            c = np.zeros((4, npts))
            c[0, :] = va / v
            c[1, :] = vb / v
            c[2, :] = vc / v
            c[3, :] = vd / v
#             print(c)
            tetras[i] = cells[np.all(c>0,axis=0)]
            #             print(c)
            bcs[i,:] = c[:,np.all(c>0,axis=0)].T
            
        return tetras,bcs
    
    def calc_bary(self,pos,tetra):
            points = self.nodes[self.tetra[tetra]]
            vap = pos - points[:, 0, :]
            vbp = pos - points[:, 1, :]
            # vcp = p - points[:, 2, :]
            # vdp = p - points[:, 3, :]
            vab = points[:, 1, :] - points[:, 0, :]
            vac = points[:, 2, :] - points[:, 0, :]
            vad = points[:, 3, :] - points[:, 0, :]
            vbc = points[:, 2, :] - points[:, 1, :]
            vbd = points[:, 3, :] - points[:, 1, :]

            va = np.sum(vbp * np.cross(vbd, vbc, axisa=1, axisb=1), axis=1) / 6.
            vb = np.sum(vap * np.cross(vac, vad, axisa=1, axisb=1), axis=1) / 6.
            vc = np.sum(vap * np.cross(vad, vab, axisa=1, axisb=1), axis=1) / 6.
            vd = np.sum(vap * np.cross(vab, vac, axisa=1, axisb=1), axis=1) / 6.
            v = np.sum(vab * np.cross(vac, vad, axisa=1, axisb=1), axis=1) / 6.
            
            c = np.zeros((4))
            c[0] = va / v
            c[1] = vb / v
            c[2] = vc / v
            c[3] = vd / v
            
            return c
    def get_tetrac(self,pos):
        aabbidx = self.pos_to_aabb_index(pos)
        # get the tetras for each aabb cell where our points are -1 are tetras which 
        # aren't seen in the cell'
        tetra_index = np.zeros(pos.shape[0]).astype(int)
        tetra_index[:] = -1
        bcs = np.zeros((pos.shape[0],4))
        cells = self.aabb[aabbidx,:]
        find_tetras(self.nodes, self.tetra, cells, pos,bcs,tetra_index)
        return tetra_index, bcs
    def get_tetra(self,pos):
        pos = np.array(pos)
        
        # create containers to store the results (tetra and the bc for each point)
        tetras = np.zeros(pos.shape[0]).astype(int)
        tetras[:] = -1
        bcs = np.zeros((pos.shape[0],4))
        
        # find the aabb cells for each point
        aabbidx = self.pos_to_aabb_index(pos)
        # get the tetras for each aabb cell where our points are -1 are tetras which 
        # aren't seen in the cell
        cells = self.aabb[aabbidx,:]
        
        # create a in array the same shape as cells array
        pos_idx = np.tile(np.arange(cells.shape[0]).T,(cells.shape[1],1)).astype(int).T
    
        # slice the pos and cells arrays for only the tetras that are inside out aabb cells
        # this will create a flat array of points and tetras
        pos2 = pos[pos_idx[cells>0]]
        cells2 = cells[cells>0]
        npts = pos2.shape[0]
        # now calculate the barycentric coordinates for all of these points
        points = self.nodes[self.tetra[cells2]]
        vap = pos2 - points[:, 0, :]
        vbp = pos2 - points[:, 1, :]
        # vcp = p - points[:, 2, :]
        # vdp = p - points[:, 3, :]
        vab = points[:, 1, :] - points[:, 0, :]
        vac = points[:, 2, :] - points[:, 0, :]
        vad = points[:, 3, :] - points[:, 0, :]
        vbc = points[:, 2, :] - points[:, 1, :]
        vbd = points[:, 3, :] - points[:, 1, :]

        va = np.sum(vbp * np.cross(vbd, vbc, axisa=1, axisb=1), axis=1) / 6.
        vb = np.sum(vap * np.cross(vac, vad, axisa=1, axisb=1), axis=1) / 6.
        vc = np.sum(vap * np.cross(vad, vab, axisa=1, axisb=1), axis=1) / 6.
        vd = np.sum(vap * np.cross(vab, vac, axisa=1, axisb=1), axis=1) / 6.
        v = np.sum(vab * np.cross(vac, vad, axisa=1, axisb=1), axis=1) / 6.
        
        c = np.zeros((4, npts))
        c[0, :] = va / v
        c[1, :] = vb / v
        c[2, :] = vc / v
        c[3, :] = vd / v
        mask = np.zeros(cells.shape).astype(bool)
        mask[cells>0] = np.all(c>0,axis=0)
#         mask= np.all(c>0,axis=0)
#         print(mask[0])

#         print('pi',pos_idx[cells>0][np.all(c>0,axis=0)])
        tetras[np.sum(mask,axis=1).astype(bool)] = cells[mask]
#         bc[np.sum(mask,axis=1).astype(bool),:] = c[:,mask].T
        return tetras#,bc

#         return self.aabb[aabbidx,:]
    def pos_to_aabb_index(self,pos):
        ix = pos[:, 0] - self.origin[None, 0]
        iy = pos[:, 1] - self.origin[None, 1]
        iz = pos[:, 2] - self.origin[None, 2]
        ix = ix // self.step[None, 0]
        iy = iy // self.step[None, 1]
        iz = iz // self.step[None, 2]
        return ix.astype(int)+iy.astype(int)*self.nx+iz.astype(int)*self.nx*self.ny
    
    def global_aabb_index_to_local(self,global_index):
        global_index = np.array(global_index)
        x_index = global_index % self.nx
        y_index = global_index // self.nx % \
                  self.ny
        z_index = global_index // self.nx // \
                  self.ny
        
        return x_index, y_index, z_index
#         corner_pos = self.origin[:,None]+self.step[:,None]*np.vstack([xcorners,ycorners,zcorners])
#         print(corner_pos)

In [5]:
def build(origin,maximum,n_tetra,nx,ny,nz):
    minx = origin[0]
    miny = origin[1]
    minz = origin[2]
    maxx = maximum[0]
    maxy =  maximum[1]
    maxz =  maximum[2]
    points = np.array([
        (minx, miny, minz),
        (maxx, miny, minz),
        (maxx, maxy, minz),
        (minx, maxy, minz),
        (minx, miny, maxz),
        (maxx, miny, maxz),
        (maxx, maxy, maxz),
        (minx, maxy, maxz)
    ])
#         self.surfaces['top'] = (
#             (minx, miny, maxz), (maxx, miny, maxz), (maxx, maxy, maxz),
#             (minx, maxy, maxz))
#         self.surfaces['bottom'] = (
#             (minx, miny, minz), (maxx, miny, minz), (maxx, maxy, minz),
#             (minx, maxy, minz))

#         self.points = points
#         # calculate the 3 principal components to find the local coordinate
#         # system
#         self.pca.fit(points)
#         # project the points into this coordinate system
#         newp = self.pca.transform(points)

    lengthU = maxx - minx
    lengthV = maxy - miny
    lengthW = maxz - minz
    boxVol = lengthU * lengthW * lengthV
    correction_factor = 1.91
    maxvol = correction_factor * boxVol / n_tetra
    facets = [
        [0, 1, 2, 3],
        [4, 5, 6, 7],
        [0, 4, 5, 1],
        [1, 5, 6, 2],
        [2, 6, 7, 3],
        [3, 7, 4, 0]
    ]

    # create the mesh
    info = meshpy.tet.MeshInfo()
    # use the projected points to build the mesh
    info.set_points(points)
    info.set_facets(facets)
    meshpy_mesh = meshpy.tet.build(info, max_volume=maxvol,
                                   options=meshpy.tet.Options('pqn'))
    nodes = np.array(meshpy_mesh.points)
    elements=  np.array(meshpy_mesh.elements, dtype=np.int64)
    neighbours = np.array(meshpy_mesh.neighbors, dtype=np.int64)
    return TetMesh2(origin,maximum,nodes,elements,neighbours,nx,ny,nz)#    def __init__(self,origin,maximum,nodes,tetra,neighbours,nx=10,ny=10,nz=10):


In [6]:
# for i,t in enumerate([100,1000,5000,7000,10000,15000,20000]):#range(1,10):
#     i=i+2
#     print(i*i*i)
x = np.linspace(0.01,.99,20)
y = np.linspace(0.01,.99,20)
z = np.linspace(0.01,.99,20)
xx,yy,zz = np.meshgrid(x,y,z)
points = np.vstack([xx.flatten(),yy.flatten(),zz.flatten()]).T
mesh2 = build([0,0,0],[1,1,1],1000,10,10,10)
#points = np.mean(mesh.nodes[mesh.tetra],axis=1)
#mask = mesh.get_tetra2(points)#np.random.random((100,3)))
# for t in mesh.tetra:
#     points = mesh.nodes[t]
#     try:
#         mesh.get_tetra([np.mean(points,axis=0)])
#     except:
#         print('exception ',i,np.mean(points,axis=0))
#mesh.get_tetra(np.array([[0.69,0.5,.99]]))#np.random.random((100,3)))
# for i in range(100):
#     print(A[i,A[i,:]>0])

[0.1 0.1 0.1]
(1000, 1495)


In [7]:
print(mesh2.aabb[:,0])

[  17   58  123   84   84   99  103   78  412   78  113   40  123   40
  115  318 1376  618  563   87  239   87  256  130   80  130   80   45
  765   45   51   40   51   40  408   49   49   45  640   45  189 1385
  746  746   80  335   80  169 1377   30   30  525   51  280  280  335
  408  122  635   33   33  127  219  278  186   49   35 1117  257   30
   30   56   56  138   82   82  209  157  546  157  315 1319  127  386
 1013  704  980  752  818   59   57   56   56  138   82    8   35    8
  546   35  154  166  415  155  155    8   31    8  597  103  349  263
  544  263  559   31   31   71  597  246  125  263  125  155   86  175
  350   86  617  349   19  906   19   83   86  175  279   86 1042  279
   85 1311   85  176  304  176  257  350  547  219   19 1163   19   67
    7  172    7  431  635  148   85  384   85   67   67  120   53  105
  635   53   59 1482  300  300    7   97    7  268 1116  510 1435 1319
 1436 1436  282  282  293  478  818   18   18   65  152   62  115   62
   14 

In [8]:
#%%snakeviz
points = np.random.random((10000,3))

In [15]:
aabbidx = mesh2.pos_to_aabb_index(points)
idx = np.arange(0,mesh2.aabb.shape[0])
xi,yi,zi = mesh2.global_aabb_index_to_local(idx[np.all(mesh2.aabb == -1,axis=1)])
# aabbidx[print(aabbidx.shape)]

In [16]:
xi

array([], dtype=int64)

In [12]:
plt.hist(nelements)

NameError: name 'nelements' is not defined

In [17]:
mask2 = mesh2.get_tetrac(points)#np.random.random((100,3)))

In [18]:
print(mask2)

(array([ 696, 1276,   -1, ...,  540,   -1,   -1]), array([[0.01196741, 0.31969509, 0.28466814, 0.38366936],
       [0.0113061 , 0.27188947, 0.19456218, 0.52224225],
       [0.        , 0.        , 0.        , 0.        ],
       ...,
       [0.31334057, 0.36676703, 0.18676215, 0.13313025],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ]]))


In [ ]:
points = mesh2.nodes[mesh2.tetra]
print(points.shape)

vab = points[ :, 1, :] - points[ :, 0, :]
vac = points[ :, 2, :] - points[ :, 0, :]
vad = points[ :, 3, :] - points[ :, 0, :]
v = np.einsum('ij, ij->i', vab, np.cross(vac, vad, axisa=1, axisb=1)) / 6.
plt.hist(v)

In [ ]:
mesh2.step[0]*mesh2.step[1]*mesh2.step[2]

In [ ]:
#%%snakeviz
mask = mesh2.get_tetra2(points)#np.random.random((100,3)))


In [ ]:
print(np.sum(mask2[0] < 0))

In [ ]:
points[mask2 == -1]

In [ ]:
np.sum(mask[0] == -1)

In [ ]:
from LoopStructural.supports.tet_mesh import TetMesh

In [ ]:
mesh = TetMesh()
mesh.setup_mesh(np.array([[0,0,0],[1,1,1]]),n_tetra=3000)

In [ ]:
mesh.elements_for_array(points)

In [8]:
xi = 8
yi = 8
zi = 8
nx=9
ny=9
nz=9

In [9]:
global_index = xi+yi*nx+zi*nx*ny

In [10]:
print(global_index)

728


In [11]:
9*9*9

729

In [ ]:
9*9*9